In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

import os
import sys
sys.path.insert(0, os.path.dirname(os.path.dirname(os.getcwd())))

os.environ['THEANO_FLAGS'] = "device=cuda1"

In [ ]:
import re
import shelve
import numpy as np
import theano
import theano.tensor as T
import lasagne
import seqtools
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial.distance import cdist

from experiments.siamese_triplet.a_data import \
    cachedir, durations, labels, recordings, \
    train_subset, val_subset
from experiments.siamese_triplet.b_preprocess import skel_feat_seqs
from experiments.siamese_triplet.c_model import skel_rnn, build_predict_fn

np.set_printoptions(linewidth=100)

In [ ]:
report = shelve.open(os.path.join(cachedir, "rnn_report"))

# Load dataset

In [ ]:
_, unique_indices = np.unique(recordings[train_subset], return_index=True)
feat_seqs_train = [
    seqtools.gather(skel_feat_seqs, train_subset[unique_indices])
]
labels_train = labels[train_subset[unique_indices]].astype(np.int32)
durations_train = durations[train_subset[unique_indices]].astype(np.int32)

feat_seqs_val = [
    seqtools.gather(skel_feat_seqs, val_subset)
]
labels_val = labels[val_subset].astype(np.int32)
durations_val = durations[val_subset].astype(np.int32)

del recordings, labels, durations, skel_feat_seqs

# Build model

In [ ]:
model_dict = skel_rnn(
    *tuple(f[0][0].shape for f in feat_seqs_train), 
    batch_size=report['meta']['batch_size'], max_time=report['meta']['max_time'], 
    encoder_kwargs=report['meta']['encoder_kwargs'])

l_linout = model_dict['l_linout']
l_in = model_dict['l_in']
l_duration = model_dict['l_duration']

last_iteration = sorted(e for e in report.keys() if re.match(r'[0-9]+', e))[-1]
lasagne.layers.set_all_param_values(l_linout, report[last_iteration]['params'])

# Evaluate performances

In [ ]:
def sample_episode(labels, voca_size, shots):
    vocabulary = np.sort(np.unique(labels))

    ep_vocabulary = np.random.choice(vocabulary, size=voca_size, replace=False)

    ep_train_subset = []
    ep_test_subset = []
    for l in ep_vocabulary:
        where_label = np.random.permutation(np.where(labels == l)[0])
        ep_train_subset.extend(where_label[:shots])
        ep_test_subset.extend(where_label[shots:])

    return np.array(ep_train_subset), np.array(ep_test_subset)


def evaluate_knn(x_train, x_test, labels_train, labels_test, k):
    ep_vocabulary = np.unique(labels_train)

    dists = cdist(x_test, x_train, metric='cosine')

    neighbours = np.argsort(dists, axis=1)[:, :k]

    neighbours_labels = labels_train[None, neighbours][0]
    neighbours_dists = dists[np.arange(len(labels_test))[:, None], neighbours]

    stats = np.empty((len(labels_test), len(ep_vocabulary)),
                     dtype=[('freq', 'i4'), ('dist_score', 'f4'), ('class', 'i4')])
    for i, l in enumerate(ep_vocabulary):
        stats['freq'][:, i] = np.sum(neighbours_labels == l, axis=1)
        stats['dist_score'][:, i] = -np.sum(neighbours_dists * (neighbours_labels == l),
                                            axis=1)
        stats['class'][:, i] = l

    stats = np.sort(stats, axis=1)

    ranks = len(ep_vocabulary) - 1 \
        - np.argmax(labels_test[:, None] == stats['class'], axis=1)

    return ranks

In [ ]:
predict_fn = build_predict_fn(model_dict, report['meta']['batch_size'], report['meta']['max_time'])
embeddings_train = predict_fn(feat_seqs_train, durations_train)
embeddings_val = predict_fn(feat_seqs_val, durations_val)

In [ ]:
shots_grid = np.array([1, 2, 3, 4, 5])
voca_size_grid = np.array([5, 10, 15, 20, 25, 30])
k = 1

train_results = np.empty((len(shots_grid), len(voca_size_grid)))
for i, shots in enumerate(shots_grid):
    for j, voca_size in enumerate(voca_size_grid):
        print("\r{}/{}".format(i, j), end='', flush=True)
        ranks = []
        for _ in range(200):
            ep_train_subset, ep_test_subset = sample_episode(labels_train, voca_size, shots)
            ep_ranks = evaluate_knn(
                embeddings_train[ep_train_subset], embeddings_train[ep_test_subset], 
                labels_train[ep_train_subset], labels_train[ep_test_subset], 
                k)
            ranks.extend(ep_ranks)
        
        train_results[i, j] = np.mean(np.array(ranks) == 0)

print("\rdone")

val_results = np.empty((len(shots_grid), len(voca_size_grid)))
for i, shots in enumerate(shots_grid):
    print("\r{}/{}".format(i, j), end='', flush=True)
    for j, voca_size in enumerate(voca_size_grid):
        ranks = []
        for _ in range(200):
            ep_train_subset, ep_test_subset = sample_episode(labels_val, voca_size, shots)
            ep_ranks = evaluate_knn(
                embeddings_val[ep_train_subset], embeddings_val[ep_test_subset],
                labels_val[ep_train_subset], labels_val[ep_test_subset], 
                k)
            ranks.extend(ep_ranks)
        
        val_results[i, j] = np.mean(np.array(ranks) == 0)

print("\rdone")

In [ ]:
fig = plt.figure(figsize=(9, 4), dpi=(100))

ax = fig.add_subplot(121, projection='3d')
ax.plot_wireframe(
    np.tile(shots_grid[:, None], (1, len(voca_size_grid))),
    np.tile(voca_size_grid[None, :], (len(shots_grid), 1)),
    train_results,
    alpha=1.)
ax.scatter(
    np.tile(shots_grid[:, None], (1, len(voca_size_grid))),
    np.tile(voca_size_grid[None, :], (len(shots_grid), 1)),
    train_results,
    c=np.ravel(train_results),
    alpha=1.)
for j in range(len(voca_size_grid)):
    ax.plot(shots_grid, train_results[:, j], zs=0, zdir='y', c='black', alpha=0.3)

ax.set_xticks(shots_grid)
ax.set_yticks(voca_size_grid)
ax.set_zlim(min(np.min(train_results), np.min(val_results)), 1)
ax.set_xlabel("shots")
ax.set_ylabel("vocabulary")
ax.set_zlabel("accuracy")
ax.set_title("training")

# ax.view_init(15, 290)
ax.view_init(20, 140)

ax = fig.add_subplot(122, projection='3d')
ax.plot_wireframe(
    np.tile(shots_grid[:, None], (1, len(voca_size_grid))),
    np.tile(voca_size_grid[None, :], (len(shots_grid), 1)),
    val_results,
    alpha=1.)
ax.scatter(
    np.tile(shots_grid[:, None], (1, len(voca_size_grid))),
    np.tile(voca_size_grid[None, :], (len(shots_grid), 1)),
    val_results,
    c=np.ravel(val_results),
    alpha=1.)
for j in range(len(voca_size_grid)):
    ax.plot(shots_grid, val_results[:, j], zs=0, zdir='y', c='black', alpha=0.3)

ax.set_xticks(shots_grid)
ax.set_yticks(voca_size_grid)
ax.set_zlim(min(np.min(train_results), np.min(val_results)), 1)
ax.set_xlabel("shots")
ax.set_ylabel("vocabulary")
ax.set_zlabel("accuracy")
ax.set_title("testing")

# ax.view_init(15, 290)
ax.view_init(20, 140)

fig.show()